# SBML to ODE for autoReduce 2
In this notebook, we will return `x`, `f`, and `P` and put it all in a function. <br>
7.15.2020 <br>
Ankita Roychoudhury


In [3]:
# imports
from libsbml import *
import sys
import numpy as np
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

**What do we want?** <br>
Input a SBML file, output `x`, `P`, and `f`. <br>
All are returned as sympy objects.

We will use the generated code function, similar to before. Except we will only return the array of functions and will not plot. We will also add the line `x,f,P = system.load_ODE_model(num_species, num_params)`.

In [13]:
def generateCodeForFile(filename):
  # 
  # read the SBML from file 
  # 
  doc = readSBMLFromFile(filename)
  if doc.getNumErrors(LIBSBML_SEV_FATAL):
    print('Encountered serious errors while reading file')
    print(doc.getErrorLog().toString())
    sys.exit(1)
    
  # clear errors
  doc.getErrorLog().clearLog()
  
  #
  # perform conversions
  #
  props = ConversionProperties()
  props.addOption("promoteLocalParameters", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  props = ConversionProperties()
  props.addOption("expandInitialAssignments", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  props = ConversionProperties()
  props.addOption("expandFunctionDefinitions", True)
  
  if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
    print('The document could not be converted')
    print(doc.getErrorLog().toString())
    
  #
  # figure out which species are variable 
  #
  mod = doc.getModel()
  variables = {}
#  print('params', mod.getNumParameters())

  for i in range(mod.getNumSpecies()): 
     species = mod.getSpecies(i)
    # if species.getBoundaryCondition() == True or variables.has_key(species.getId()):
     if species.getBoundaryCondition() == True or (species.getId() in variables.keys()):


       continue
     variables[species.getId()] = []
  #print(variables)
  
  #
  # start generating the code by appending to bytearray
  #
  # Define num_species, num_params here, will save and be used
  num_species = mod.getNumSpecies()
  num_params = mod.getNumParameters()
    
    
  generated_code = bytearray()
  generated_code.extend('from numpy import *\n'.encode('latin-1'))
  generated_code.extend('x,f,P = system.load_ODE_model(num_species, num_params)'.encode('latin-1'))
  #generated_code.extend('x = np.ones(num_species)'.encode('latin-1'))

 # generated_code.extend('from matplotlib.pylab import *\n'.encode('latin-1'))
 # generated_code.extend('from matplotlib.pyplot import *\n'.encode('latin-1'))
 # generated_code.extend('from scipy.integrate import odeint \n'.encode('latin-1'))
  
  generated_code.extend('\n'.encode('latin-1'))
#  generated_code.extend('def simulateModel(t0, tend, numPoints):\n'.encode('latin-1'))
  
  # write out compartment values 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#compartments\n'.encode('latin-1'))
  for i in range(mod.getNumCompartments()):
    element = mod.getCompartment(i)
    generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getSize()).encode('latin-1'))
  
  # write out parameter values 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#global parameters\n'.encode('latin-1'))
  for i in range(mod.getNumParameters()):
    element = mod.getParameter(i)
    generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getValue()).encode('latin-1'))
  
  # write out boundary species 
  generated_code.extend('\n'.encode('latin-1'))
  generated_code.extend('#boundary species\n'.encode('latin-1'))
  for i in range(mod.getNumSpecies()):
    element = mod.getSpecies(i)
    if element.getBoundaryCondition() == False: 
      continue 
    if element.isSetInitialConcentration(): 
      generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getInitialConcentration()).encode('latin-1'))
    else:
      generated_code.extend('{0} = {1}\n'.format(element.getId(), element.getInitialAmount()).encode('latin-1'))  
  
  # write derive function
  generated_code.extend('\n'.encode('latin-1'))
  #generated_code.extend('  def ode_fun(__Y__, t):\n'.encode('latin-1'))
    
   # Set all species equal to a __Y__[i] 
  for i in range(len(variables.keys())): 
    generated_code.extend('{0} = x[{1}]\n'.format(list(variables.keys())[i], i).encode('latin-1'))
    
  generated_code.extend('\n'.encode('latin-1'))
  
  for i in range(mod.getNumReactions()): 
    reaction = mod.getReaction(i)
    kinetics = reaction.getKineticLaw()  
    
    generated_code.extend('{0} = {1}\n'.format(reaction.getId(),  kinetics.getFormula()).encode('latin-1'))
    
    for j in range(reaction.getNumReactants()): 
      ref = reaction.getReactant(j)
      species = mod.getSpecies(ref.getSpecies())
      if species.getBoundaryCondition() == True: 
        continue
      if ref.getStoichiometry() == 1.0: 
        variables[species.getId()].append('-{0}'.format(reaction.getId()))
      else:
        variables[species.getId()].append('-({0})*{1}'.format(ref.getStoichiometry(), reaction.getId()))
    for j in range(reaction.getNumProducts()): 
      ref = reaction.getProduct(j)
      species = mod.getSpecies(ref.getSpecies())
      if species.getBoundaryCondition() == True: 
        continue
      if ref.getStoichiometry() == 1.0: 
        variables[species.getId()].append('{0}'.format(reaction.getId()))
      else:
        variables[species.getId()].append('({0})*{1}'.format(ref.getStoichiometry(), reaction.getId()))
  
  generated_code.extend('\n'.encode('latin-1'))
    
  generated_code.extend('funcs = array(['.encode('latin-1'))
 # generated_code.extend('    print( array(['.encode('latin-1'))
  for i in range(len(variables.keys())):
    for eqn in variables[list(variables.keys())[i]]:
      generated_code.extend(' + ({0})'.format(eqn).encode('latin-1'))
    if i + 1 < len(variables.keys()):
      generated_code.extend(',\n      '.encode('latin-1'))
  generated_code.extend('    ])\n'.encode('latin-1'))
  generated_code.extend('\n'.encode('latin-1'))

  return generated_code

Now define all encompassing function.

In [14]:
#def sbml_to_ode(filename):
filename = 'CRN.xml'

doc = readSBMLFromFile(filename)
props = ConversionProperties()
props.addOption("promoteLocalParameters", True)
doc.convert(props)
mod = doc.getModel()

num_species = mod.getNumSpecies()
num_params = mod.getNumParameters()

gen_code = generateCodeForFile(filename)
exec(gen_code)

Test it!

In [15]:
funcs

array([-15.0*x0*x1*x2 - 1.5*x0*x3*x4 + 1.5*x5 + 15.0*x6,
       -15.0*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 15.0*x1*x11*x8 - 1.5*x1*x21**2*x30 - 15.0*x1*x65 + 1.5*x13 + 15.0*x32 + 30.0*x44 + 1.5*x5 + 1.5*x66,
       -15.0*x0*x1*x2 + 1.5*x5,
       -1.5*x0*x3*x4 - 15.0*x3*x7 + 15.0*x6 + 1.5*x9,
       -1.5*x0*x3*x4 - 1.5*x11*x12*x4 + 15.0*x14 - 1.5*x26*x4*x65 - 15.0*x27*x30*x4 + 1.5*x31 - 30.0*x38**2*x4**2*x41 + 3.0*x43 + 15.0*x6 + 15.0*x67,
       15.0*x0*x1*x2 - 36001.5*x5, 1.5*x0*x3*x4 + 36000.0*x5 - 15.0*x6,
       15.0*x10 - 15.0*x3*x7 - 1.5*x7*x8 + 1.5*x9,
       -15.0*x1*x11*x8 + 15.0*x10 + 1.5*x13 - 1.5*x7*x8,
       15.0*x3*x7 - 36001.5*x9, -15.0*x10 + 1.5*x7*x8 + 36000.0*x9,
       -15.0*x1*x11*x8 - 1.5*x11*x12*x4 + 1.5*x13 + 15.0*x14,
       -1.5*x11*x12*x4 - 15.0*x12*x15 + 15.0*x14 + 1.5*x17,
       15.0*x1*x11*x8 - 36001.5*x13,
       1.5*x11*x12*x4 + 36000.0*x13 - 15.0*x14,
       -15.0*x12*x15 - 1.5*x15*x16**2 + 1.5*x17 + 15.0*x18,
       -3.0*x15*x16**2 - 30.0*x16**2*x19*x20

In [16]:
funcs_change = array([-15.0*x0*x1*x2 - 1.5*x0*x3*x4 + 1.5*x5 + 15.0*x6,
       -15.0*x0*x1*x2 - 3.0*x1**2*x41*x42**2 - 15.0*x1*x11*x8 - 1.5*x1*x21**2*x30 - 15.0*x1*x65 + 1.5*x13 + 15.0*x32 + 30.0*x44 + 1.5*x5 + 1.5*x66,
       -15.0*x0*x1*x2 + 1.5*x5,
       -1.5*x0*x3*x4 - 15.0*x3*x7 + 15.0*x6 + 1.5*x9,
       -1.5*x0*x3*x4 - 1.5*x11*x12*x4 + 15.0*x14 - 1.5*x26*x4*x65 - 15.0*x27*x30*x4 + 1.5*x31 - 30.0*x38**2*x4**2*x41 + 3.0*x43 + 15.0*x6 + 15.0*x67,
       15.0*x0*x1*x2 - 36001.5*x5, 1.5*x0*x3*x4 + 36000.0*x5 - 15.0*x6,
       15.0*x10 - 15.0*x3*x7 - 1.5*x7*x8 + 1.5*x9,
       -15.0*x1*x11*x8 + 15.0*x10 + 1.5*x13 - 1.5*x7*x8,
       15.0*x3*x7 - 36001.5*x9, -15.0*x10 + 1.5*x7*x8 + 36000.0*x9,
       -15.0*x1*x11*x8 - 1.5*x11*x12*x4 + 1.5*x13 + 15.0*x14,
       -1.5*x11*x12*x4 - 15.0*x12*x15 + 15.0*x14 + 1.5*x17,
       15.0*x1*x11*x8 - 36001.5*x13,
       1.5*x11*x12*x4 + 36000.0*x13 - 15.0*x14,
       -15.0*x12*x15 - 1.5*x15*x16**2 + 1.5*x17 + 15.0*x18,
       -3.0*x15*x16**2 - 30.0*x16**2*x19*x20**2 - 30.0*x16**2*x20**2*x25*x26 + 30.0*x18 + 3.0*x23 + 3.0*x28,
       15.0*x12*x15 - 36001.5*x17,
       1.5*x15*x16**2 + 36000.0*x17 - 15.0*x18,
       -15.0*x16**2*x19*x20**2 - 1.5*x19*x21**2*x22**2 + 1.5*x23 + 15.0*x24,
       -30.0*x16**2*x19*x20**2 - 30.0*x16**2*x20**2*x25*x26 - 1.5*x20*x49*x50 - 1.5*x20*x61*x62 + 3.0*x23 + 3.0*x28 + 15.0*x52 + 15.0*x64,
       -3.0*x1*x21**2*x30 - 3.0*x19*x21**2*x22**2 - 30.0*x21**2*x33 + 30.0*x24 + 30.0*x32 + 3.0*x35,
       -3.0*x19*x21**2*x22**2 - 3.0*x22**2*x25*x27 - 15.0*x22*x46*x49 - 15.0*x22*x58*x61 + 30.0*x24 + 30.0*x29 + 1.5*x51 + 1.5*x63,
       15.0*x16**2*x19*x20**2 - 36001.5*x23,
       1.5*x19*x21**2*x22**2 + 36000.0*x23 - 15.0*x24,
       -15.0*x16**2*x20**2*x25*x26 - 1.5*x22**2*x25*x27 + 1.5*x28 + 15.0*x29,
       -15.0*x16**2*x20**2*x25*x26 - 1.5*x26*x4*x65 + 1.5*x28 + 15.0*x67,
       -1.5*x22**2*x25*x27 - 15.0*x27*x30*x4 + 15.0*x29 + 1.5*x31,
       15.0*x16**2*x20**2*x25*x26 - 36001.5*x28,
       1.5*x22**2*x25*x27 + 36000.0*x28 - 15.0*x29,
       -1.5*x1*x21**2*x30 - 15.0*x27*x30*x4 + 1.5*x31 + 15.0*x32,
       15.0*x27*x30*x4 - 36001.5*x31,
       1.5*x1*x21**2*x30 + 36000.0*x31 - 15.0*x32,
       -15.0*x21**2*x33 - 1.5*x33*x34**2 + 1.5*x35 + 15.0*x36,
       -3.0*x33*x34**2 - 30.0*x34**2*x37 + 30.0*x36 + 3.0*x39,
       15.0*x21**2*x33 - 36001.5*x35,
       1.5*x33*x34**2 + 36000.0*x35 - 15.0*x36,
       -15.0*x34**2*x37 - 1.5*x37*x38**2 + 1.5*x39 + 15.0*x40,
       -3.0*x37*x38**2 - 30.0*x38**2*x4**2*x41 + 30.0*x40 + 3.0*x43,
       15.0*x34**2*x37 - 36001.5*x39,
       1.5*x37*x38**2 + 36000.0*x39 - 15.0*x40,
       -1.5*x1**2*x41*x42**2 - 15.0*x38**2*x4**2*x41 + 1.5*x43 + 15.0*x44,
       -3.0*x1**2*x41*x42**2 - 30.0*x42**2*x45 + 30.0*x44 + 3.0*x47,
       15.0*x38**2*x4**2*x41 - 36001.5*x43,
       1.5*x1**2*x41*x42**2 + 36000.0*x43 - 15.0*x44,
       -15.0*x42**2*x45 - 1.5*x45*x46 + 1.5*x47 + 15.0*x48,
       -15.0*x22*x46*x49 - 1.5*x45*x46 + 15.0*x48 + 1.5*x51,
       15.0*x42**2*x45 - 36001.5*x47,
       1.5*x45*x46 + 36000.0*x47 - 15.0*x48,
       -1.5*x20*x49*x50 - 15.0*x22*x46*x49 + 1.5*x51 + 15.0*x52,
       -1.5*x20*x49*x50 - 15.0*x50*x53 + 15.0*x52 + 1.5*x55,
       15.0*x22*x46*x49 - 36001.5*x51,
       1.5*x20*x49*x50 + 36000.0*x51 - 15.0*x52,
       -15.0*x50*x53 - 1.5*x53*x54 + 1.5*x55 + 15.0*x56,
       -1.5*x53*x54 - 15.0*x54*x57 + 15.0*x56 + 1.5*x59,
       15.0*x50*x53 - 36001.5*x55, 1.5*x53*x54 + 36000.0*x55 - 15.0*x56,
       -15.0*x54*x57 - 1.5*x57*x58 + 1.5*x59 + 15.0*x60,
       -15.0*x22*x58*x61 - 1.5*x57*x58 + 15.0*x60 + 1.5*x63,
       15.0*x54*x57 - 36001.5*x59, 1.5*x57*x58 + 36000.0*x59 - 15.0*x60,
       -1.5*x20*x61*x62 - 15.0*x22*x58*x61 + 1.5*x63 + 15.0*x64,
       -1.5*x20*x61*x62 + 15.0*x64, 15.0*x22*x58*x61 - 36001.5*x63,
       1.5*x20*x61*x62 + 36000.0*x63 - 15.0*x64,
       -15.0*x1*x65 - 1.5*x26*x4*x65 + 1.5*x66 + 15.0*x67,
       15.0*x1*x65 - 3.5*x66, 1.5*x26*x4*x65 + 2.0*x66 - 15.0*x67],
      dtype=object)

NameError: name 'x0' is not defined

In [18]:
g = funcs
g_func = lambda val: numpy.array( g.subs( {x:val} ).tolist(), dtype=float )


TypeError: 'function' object is not subscriptable

In [17]:
np.array(funcs[0]).astype(np.float64)

TypeError: can't convert expression to float

In [15]:
np.array(funcs[0], dtype = np.float64)

TypeError: can't convert expression to float

In [5]:
x[0]

x0

In [6]:
P[0]

P0

In [8]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,auto_reduce,sympy,libsbml,sys,jupyterlab

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
auto_reduce 0.1
sympy 1.5.1
libsbml 5.18.0
sys 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
jupyterlab 1.2.6
